# Create a smooth fiducial for SOLikeT/lensing

In this notebook, we generate a fiducial smooth (i.e., binned spectrum-based theoretical prediction, no map-based random realization) dataset. We start from lensing spectra (lensing).

In [ ]:
import os

import camb
import cobaya
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from cobaya.tools import resolve_packages_path

print("     Numpy :", np.__version__)
print("Matplotlib :", mpl.__version__)
print("      CAMB :", camb.__version__)
print("    Cobaya :", cobaya.__version__)

In [ ]:
# Fiducial cosmo params. When used in a MCMC, the smooth data must agree
# with this input values

cosmo_params = {
    "cosmomc_theta": 0.0104090,
    "logA": {"value": 3.045, "drop": True},
    "As": {"value": "lambda logA: 1e-10*np.exp(logA)"},
    "ombh2": 0.02236,
    "omch2": 0.1202,
    "ns": 0.9649,
    "Alens": 1.0,
    "tau": 0.0544,
    "mnu": 0.06,
    "nnu": 3.044,
}

Let's use SOLikeT/lensing to generate the fiducial spectra

In [ ]:
custom_packages_path = None

# Try to resolve global path
if custom_packages_path is None:
    packages_path = resolve_packages_path()
else:
    packages_path = custom_packages_path

In [ ]:
info = {
    "likelihood": {
        "soliket.LensingLikelihood": {"theory_lmax": 5000, "stop_at_error": False}
    },
    "theory": {
        "camb": {
            "extra_args": {
                "bbn_predictor": "PArthENoPE_880.2_standard.dat",
                "WantTransfer": True,
                "Transfer.high_precision": True,
                "Transfer.kmax": 1.2,
                "halofit_version": "mead",
                "lens_potential_accuracy": 1,
                "num_massive_neutrinos": 1,
                "theta_H0_range": [20, 100],
            },
            "stop_at_error": True,
        }
    },
    "params": cosmo_params,
    "packages_path": packages_path,
    "debug": False,
}

In [ ]:
from cobaya.model import get_model

model = get_model(info)

In [ ]:
# model.components
my_lensing = model.likelihood["soliket.LensingLikelihood"]
# chi2 = -2 * (model.loglikes(fg_params)[0] - my_mflike.logp_const)
model.loglikes()

Let's have a look at the comparison between simulated noisy data and the smooth theory from the fiducial parms above. Note that the function ``_get_data`` from ``my_lensing`` takes the input ell-by-ell Cls from camb and bin and correct them to be compared against data.

In [ ]:
# Just inspecting the diagonal part of the covariance
np.diag(my_lensing.data.cov)

In [ ]:
# Plotting data against smooth theory
plt.plot(
    my_lensing.x,
    my_lensing._get_theory(**cosmo_params),
    ls=None,
    marker="o",
)
plt.errorbar(
    my_lensing.x,
    my_lensing.y,
    yerr=np.sqrt(np.diag(my_lensing.data.cov)),
    marker="s",
    mfc="red",
    mec="green",
    ms=1,
    mew=1,
)
plt.xscale("log")

We will now replace the noisy data above (in orange) with the binned smooth theory (in blue). We will make use of the astropy functions to read and write fits file. Let's access the lensing data file.

In [ ]:
import sacc

sacc_data = sacc.Sacc.load_fits(
    packages_path + "/data/LensingLikelihood/lensing.sacc.fits"
)

ell, clkk, cov, ind = sacc_data.get_ell_cl(
    "cl_00", "ck", "ck", return_cov=True, return_ind=True
)
wins = sacc_data.get_bandpower_windows(ind)

print(sacc_data.metadata)
print(sacc_data.tracers)

Let's have a look at the data field

In [ ]:
for d in sacc_data.data:
    print(d)

Ok! Let's now create a new sacc file with the new theory-based data.

In [ ]:
s = sacc.Sacc()
s.metadata["info"] = "CMB lensing power spectra from reconstruction simulations"

beam = np.ones_like(ell)

s.add_tracer(
    tracer_type="Map",
    name="ck",
    quantity="cmb_convergence",
    spin=0,
    map_unit="uK_CMB",
    ell=ell,
    beam=beam,
)

s.add_ell_cl(
    "cl_00",  # Data type
    "ck",  # 1st tracer's name
    "ck",  # 2nd tracer's name
    ell,  # Effective multipole
    my_lensing._get_theory(**cosmo_params),  # Power spectrum values
    window=wins,  # Bandpower windows
)

cov = sacc.covariance.FullCovariance.from_hdu(cov)
s.add_covariance(cov, overwrite=False)

ndir = (
    packages_path
    + "/data/LensingLikelihood/soliket_lensing/smooth_data/"
    + "clkk_reconstruction_sim_smooth.fits"
)

os.makedirs(
    packages_path + "/data/LensingLikelihood/soliket_lensing/smooth_data/",
    exist_ok=True,
)
s.save_fits(ndir, overwrite=True)

# Test new data

Let's read the newly created data and check that we get chi2=0 if compared against same fiducial

In [ ]:
info = {
    "likelihood": {
        "soliket.LensingLikelihood": {
            # "data_folder": namedir,
            "data_filename": "soliket_lensing/smooth_data/"
            "clkk_reconstruction_sim_smooth.fits",
            "theory_lmax": 5000,
            "stop_at_error": False,
        }
    },
    "theory": {
        "camb": {
            "extra_args": {
                "bbn_predictor": "PArthENoPE_880.2_standard.dat",
                "WantTransfer": True,
                "Transfer.high_precision": True,
                "Transfer.kmax": 1.2,
                "halofit_version": "mead",
                "lens_potential_accuracy": 1,
                "num_massive_neutrinos": 1,
                "theta_H0_range": [20, 100],
            },
            "stop_at_error": True,
        }
    },
    "params": cosmo_params,
    "packages_path": packages_path,
    "debug": False,
}

In [ ]:
from cobaya.model import get_model

model_test = get_model(info)

In [ ]:
my_lensing_test = model_test.likelihood["soliket.LensingLikelihood"]
model_test.loglikes()

Let's have a qualitative look at the comparison between smooth data and binned theory (obtained from the very same fiducial params). They should coincide!

In [ ]:
plt.plot(
    my_lensing_test.x,
    my_lensing_test._get_theory(**cosmo_params),
    ls=None,
    marker="o",
)
plt.errorbar(
    my_lensing_test.x,
    my_lensing_test.y,
    yerr=np.sqrt(np.diag(my_lensing_test.data.cov)),
    marker="s",
    mfc="red",
    mec="green",
    ms=1,
    mew=1,
)
plt.xscale("log")

Ok! Let's now quantitatively check that data and theory are the same. We can do that by taking the difference between data and theory, and by computing the chi2 part of the loglike. Both should be zero!

In [ ]:
my_lensing_test._get_theory(**cosmo_params) - my_lensing_test.y

In [ ]:
delta = my_lensing_test._get_theory(**cosmo_params) - my_lensing_test._get_data()[1]
-0.5 * np.dot(delta, np.linalg.inv(my_lensing_test._get_cov()).dot(delta))

Awesome!!!